In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.set_num_threads(8)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",
            'random_class_idx' : False,
            'method': "",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 50,
            'per_task_examples': np.inf,
            # 'per_task_examples': 10000,
            'per_task_memory_examples': 32,
            'batch_size_train': 256,
            'batch_size_memory': 256,
            'batch_size_validation': 256,
            'tau': 1,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'learning_rate_decay_epoch': [30, 50, 70, 90],
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:5' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001,
            'lambda': .01,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR10/seed=10_epoch=50_lr=0.01_alpha=0.001_tau=1


In [2]:
from datasets import MNIST, FashionMNIST, BiasedMNIST, CIFAR10, CIFAR100
if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                joint = (params['method'] == "joint"),
                                random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 28, 28)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)


[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
benchmark.trains[1].true_index

array([    4,     5,    29, ..., 49994, 49998, 49999])

In [4]:
torch.get_num_threads()

8

In [5]:
idx = 8

In [6]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

from backbones.resnet import ResNet18


# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

backbone = ResNet18(
    input_dim=input_dim, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

modify resnet for cifar


In [19]:
backbone.fc.weight.data

tensor([[-0.0237, -0.0204, -0.0312,  ...,  0.0054,  0.0166, -0.0270],
        [ 0.0196, -0.0321, -0.0202,  ...,  0.0272, -0.0204,  0.0196],
        [ 0.0281,  0.0191,  0.0085,  ..., -0.0004, -0.0383, -0.0207],
        ...,
        [-0.0283,  0.0079,  0.0365,  ..., -0.0317, -0.0030, -0.0255],
        [-0.0402,  0.0383,  0.0357,  ..., -0.0319,  0.0149,  0.0105],
        [-0.0219,  0.0396,  0.0271,  ..., -0.0411,  0.0217,  0.0155]],
       device='cuda:5')

In [20]:
weights = backbone.fc.weight.data

In [23]:
weights[0] = torch.nn.Parameter(weights[0] +1)
weights

tensor([[ 9.7627e-01,  9.7964e-01,  9.6882e-01,  ...,  1.0054e+00,
          1.0166e+00,  9.7298e-01],
        [ 1.9583e-02, -3.2069e-02, -2.0166e-02,  ...,  2.7164e-02,
         -2.0435e-02,  1.9625e-02],
        [ 2.8105e-02,  1.9145e-02,  8.4660e-03,  ..., -3.9455e-04,
         -3.8273e-02, -2.0749e-02],
        ...,
        [-2.8313e-02,  7.9134e-03,  3.6549e-02,  ..., -3.1680e-02,
         -2.9774e-03, -2.5549e-02],
        [-4.0180e-02,  3.8316e-02,  3.5739e-02,  ..., -3.1938e-02,
          1.4906e-02,  1.0527e-02],
        [-2.1864e-02,  3.9625e-02,  2.7119e-02,  ..., -4.1076e-02,
          2.1699e-02,  1.5526e-02]], device='cuda:5', grad_fn=<CopySlices>)

In [24]:
backbone.fc.weight.data

tensor([[ 9.7627e-01,  9.7964e-01,  9.6882e-01,  ...,  1.0054e+00,
          1.0166e+00,  9.7298e-01],
        [ 1.9583e-02, -3.2069e-02, -2.0166e-02,  ...,  2.7164e-02,
         -2.0435e-02,  1.9625e-02],
        [ 2.8105e-02,  1.9145e-02,  8.4660e-03,  ..., -3.9455e-04,
         -3.8273e-02, -2.0749e-02],
        ...,
        [-2.8313e-02,  7.9134e-03,  3.6549e-02,  ..., -3.1680e-02,
         -2.9774e-03, -2.5549e-02],
        [-4.0180e-02,  3.8316e-02,  3.5739e-02,  ..., -3.1938e-02,
          1.4906e-02,  1.0527e-02],
        [-2.1864e-02,  3.9625e-02,  2.7119e-02,  ..., -4.1076e-02,
          2.1699e-02,  1.5526e-02]], device='cuda:5')

In [ ]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [ ]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

In [ ]:
metric_manager_callback.meters['EER'].get_eer()

In [ ]:
metric_manager_callback.meters['std'].get_std()

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

In [ ]:
metric_manager_callback.meters['accuracy'].compute_overall()

In [ ]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")
